In [2]:
import nncf
import torch

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, onnx, openvino


In [ ]:
import nncf
import torch

calibration_loader = torch.utils.data.DataLoader(...)

def transform_fn(data_item):
    images, _ = data_item
    #return images  torch
    return images.numpy()
    
    
calibration_dataset = nncf.Dataset(calibration_loader, transform_fn)

In [3]:
import torchvision
model = torchvision.models.resnet50(pretrained=True)
#这个还是需要转化
#quantized_model = nncf.quantize(model, calibration_dataset)

KeyboardInterrupt: 

In [ ]:
# compile the model to transform quantized operations to int8
model_int8 = ov.compile_model(quantized_model)

input_fp32 = ... # FP32 model input
res = model_int8(input_fp32)

# save the model
ov.save_model(quantized_model, "quantized_model.xml")

In [ ]:
from openvino.runtime import Core
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
import numpy as np
import openvino as ov
import urllib.request
urllib.request.urlretrieve(
    url='https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/main/notebooks/utils/notebook_utils.py',
    filename='notebook_utils.py'
)

from notebook_utils import download_file

import numpy as np
import openvino as ov
import torch
import IPython.display as ipd

from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

from pathlib import Path

# Set the data and model directories, model source URL and model filename.
MODEL_DIR = Path("model")
MODEL_DIR.mkdir(exist_ok=True)

In [ ]:
torch_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h", ctc_loss_reduction="mean")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
BATCH_SIZE = 1
MAX_SEQ_LENGTH = 30480
ov_model = ov.convert_model(torch_model, example_input=torch.zeros([1, MAX_SEQ_LENGTH], dtype=torch.float))

ir_model_path = MODEL_DIR / "wav2vec2_base.xml"
ov.save_model(ov_model, ir_model_path)

In [ ]:
from datasets import load_dataset


dataset = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")
test_sample = dataset[0]["audio"]


# define preprocessing function for converting audio to input values for model
def map_to_input(batch):
    preprocessed_signal = processor(batch["audio"]["array"], return_tensors="pt", padding="longest", sampling_rate=batch['audio']['sampling_rate'])
    input_values = preprocessed_signal.input_values
    batch['input_values'] = input_values
    return batch


# apply preprocessing function to dataset and remove audio column, to save memory as we do not need it anymore
dataset = dataset.map(map_to_input, batched=False, remove_columns=["audio"])

In [ ]:
import nncf
from nncf.parameters import ModelType

def transform_fn(data_item):

    return np.array(data_item["input_values"])


calibration_dataset = nncf.Dataset(dataset, transform_fn)

quantized_model = nncf.quantize(
    ov_model,
    calibration_dataset,
    model_type=ModelType.TRANSFORMER,  # specify additional transformer patterns in the model
    ignored_scope=nncf.IgnoredScope(
        names=[
            "__module.wav2vec2.feature_extractor.conv_layers.1.conv/aten::_convolution/Convolution",
            "__module.wav2vec2.feature_extractor.conv_layers.2.conv/aten::_convolution/Convolution",
            "__module.wav2vec2.feature_extractor.conv_layers.3.conv/aten::_convolution/Convolution",
            "__module.wav2vec2.feature_extractor.conv_layers.0.conv/aten::_convolution/Convolution",
        ],
    ),
)

In [ ]:
core=ov.Core()
compiled_model = core.compile_model(model=quantized_model, device_name='CPU')

input_data = np.expand_dims(test_sample["array"], axis=0)

In [ ]:
predictions = compiled_model(input_data)[0]
predicted_ids = np.argmax(predictions, axis=-1)
transcription = processor.batch_decode(torch.from_numpy(predicted_ids))
print(transcription)

# Weight Compression
    只针对模型的权重，
    enables inference of exceptionally large models that cannot be accommodated in the device memory;

    reduces storage and memory overhead, making models more lightweight and less resource intensive for deployment;

    improves inference speed by reducing the latency of memory access when computing the operations with weights, for example, Linear layers. The weights are smaller and thus faster to load from memory;

    unlike quantization, does not require sample data to calibrate the range of activation values.

#### 8-bit weight quantization

In [ ]:
from nncf import compress_weights

model = compress_weights(model) # model is openvino.Model object


##### 4-bit weight quantization 

In [ ]:
from nncf import compress_weights, CompressWeightsMode

# Example: Compressing weights with INT4_ASYM mode, group size of 64, and 90% INT4 ratio
compressed_model = compress_weights(
  model,
  mode=CompressWeightsMode.INT4_ASYM,
  group_size=64,
  ratio=0.9,
)